<h1 align="center">Cleaning and Preparing Places Data</h1>

*******************************************************************************************************************************

<h2>1. Introduction</h2>

### Data Cleaning Overview

In this project, data was collected using the `fetch_places.py` script from the Google Places API, focusing on places in Santo André and the surrounding areas. Due to the limitations in the API’s collection radius and number of requests, grid points were used as a strategy to cover the entire area. By dividing the region into a grid, the script systematically collected data from each point, ensuring broader coverage of the city and its surroundings. The data was then cleaned to prepare it for analysis. The key steps involved:

- **Handling missing values**: Removed rows with missing ratings or review counts.
- **Removing duplicates**: Eliminated duplicate entries based on the Place ID.
- **Standardizing addresses**: Split and cleaned the address field into separate columns for street, neighborhood, and city.
- **Excluding irrelevant places**: Filtered out places that didn’t match pet shop-related keywords.
- **City filtering**: Optionally filtered the dataset to include only places in Santo André.

These steps ensured the data is clean, accurate, and ready for further analysis.

<h2>2. Initialization</h2>

In [ ]:
# Library Imports
import csv
import pandas as pd
import numpy as np
import os
import re

In [ ]:
# Configures Pandas display settings: shows all columns in DataFrames and suppresses chained assignment warnings  
pd.set_option("display.max_columns", None)  
pd.options.mode.chained_assignment = None  

<h2>3. Load the Dataset</h2>

In [ ]:
PATH = os.path.abspath(os.path.join("..", "data", "raw", "places_raw.csv"))

In [ ]:
places_df = pd.read_csv(PATH, sep=";", header=0, encoding="utf-8", dtype={"Number of Reviews": "Int64"})

In [ ]:
places_df.shape

In [ ]:
places_df.columns

In [ ]:
places_df.head(3)

<h2>4. Data Cleaning</h2>

<h3>4.1 Address Splitting and Reorganization</h3>

In [ ]:
def split_address(address):
    """
    Splits an address string into its components: street, neighborhood, and city.
    
    The function attempts to match several patterns:
    
    1. Full pattern: "Street Name, Number - Neighborhood, City"
       - If matched, it extracts and combines the street name and number,
         and then extracts the neighborhood and city.
    
    2. Alternative pattern: "Street - Neighborhood, City"
       - If matched, it extracts the street, neighborhood, and city directly.
    
    3. If the address does not contain a dash but contains a comma,
       it is assumed to be in the format "Neighborhood, City".
       - In this case, the street is set to an empty string.
    
    If none of the above patterns match, the function returns the full address 
    in the 'Street' field and empty strings for 'Neighborhood' and 'City'.
    
    Parameters:
        address (str): The address string to be split.
    
    Returns:
        pd.Series: A Pandas Series with three elements:
                   [street, neighborhood, city].
    """
    address = str(address)
    
    # Full pattern: "Street Name, Number - Neighborhood, City"
    match = re.match(r"(.+), (\d+) - (.+), (.+)", address)
    if match:
        street = f"{match.group(1)}, {match.group(2)}"  # Combine street name and number
        neighborhood = match.group(3)
        city = match.group(4)
        return pd.Series([street, neighborhood, city])
    
    # Alternative pattern: "Street - Neighborhood, City"
    match_simple = re.match(r"(.+?) - (.+), (.+)", address)
    if match_simple:
        street = match_simple.group(1)
        neighborhood = match_simple.group(2)
        city = match_simple.group(3)
        return pd.Series([street, neighborhood, city])
    
    # If no dash is present but a comma exists, assume format "Neighborhood, City"
    if " - " not in address and "," in address:
        parts = [part.strip() for part in address.split(",", 1)]
        if len(parts) == 2:
            neighborhood, city = parts
            return pd.Series(["", neighborhood, city])
    
    # Fallback: return full address in 'Street' and empty 'Neighborhood' and 'City'
    return pd.Series([address, "", ""])

In [ ]:
# Apply the function and create the new columns
places_df[["Street", "Neighborhood", "City"]] = places_df["Address"].apply(split_address)

In [ ]:
# Drop the original Address column
places_df = places_df.drop(columns=["Address"])

In [ ]:
# Define the order of the columns
column_order = ["Name", "Street", "Neighborhood", "City"]

# Add the remaining columns, excluding 'Name', 'Street', 'Neighborhood', and 'City'
remaining_columns = [col for col in places_df.columns if col not in column_order]

# Combine the defined columns with the remaining ones
cols = column_order + remaining_columns

# Reorder the DataFrame columns
places_df = places_df[cols]

In [ ]:
places_df.head(5)

In [ ]:
def check_empty_address_fields(df):
    """
    Checks for rows where the 'Street', 'Neighborhood', or 'City' fields are empty or contain only whitespace.
    
    This function filters the DataFrame to find any rows with empty address fields and returns:
      - count (int): The total number of such rows.
      - sample (pd.DataFrame): A DataFrame containing the first 5 rows with empty address fields.
    
    Parameters:
        df (pd.DataFrame): DataFrame containing the columns 'Street', 'Neighborhood', and 'City'.
        
    Returns:
        tuple: A tuple (count, sample) where:
            count (int): Number of rows with any empty address field.
            sample (pd.DataFrame): The first 5 rows from the filtered DataFrame.
    """
    # Create a mask to identify rows with empty or blank fields in 'Street', 'Neighborhood', or 'City'
    mask = (
        (df["Street"].isnull() | (df["Street"].str.strip() == "")) |
        (df["Neighborhood"].isnull() | (df["Neighborhood"].str.strip() == "")) |
        (df["City"].isnull() | (df["City"].str.strip() == ""))
    )
    
    # Filter the DataFrame using the mask
    empty_df = df[mask]
    
    # Count the number of rows with empty fields
    count = empty_df.shape[0]
    
    # Get the first 5 rows of the filtered DataFrame as a sample
    sample = empty_df.head(5)
    
    return count, sample

In [ ]:
# Call the function to check empty address fields in 'places_df'
count, sample = check_empty_address_fields(places_df)

In [ ]:
# Print the total number of rows with empty address fields
print(f"Total rows with empty address fields: {count}")

In [ ]:
# Display the first 5 rows with empty address fields
sample

<h3>4.2 Identifying and Removing Duplicate Places</h3>

In [ ]:
# Find duplicate places based on the 'Place ID' column
duplicates = places_df[places_df.duplicated(subset="Place ID", keep="first")]

In [ ]:
# Count the number of duplicate places
num_duplicates = duplicates.shape[0]
print(f"Number of duplicate places: {num_duplicates}")

In [ ]:
# Remove duplicate places, keeping only the first occurrence based on 'Place ID'
places_df.drop_duplicates(subset="Place ID", inplace=True, keep="first")

<h3>4.3 Handling Missing Data</h3>

In [ ]:
# Count the missing values in each column
places_df.isnull().sum()

In [ ]:
# Remove places with missing 'Rating' or 'Number of Reviews'
places_df.dropna(subset=["Rating", "Number of Reviews"], inplace=True)

<h3>4.4 Standardizing Place Categories</h3>

In [ ]:
# Count the occurrences of each place type in the DataFrame  
places_df["Type"].value_counts()  

In [ ]:
# Standardize place category names by mapping them to a consistent format
category_mapping = {
    "Pet store": "pet_store",
    "Veterinary care": "veterinary_care"
}
places_df["Type"] = places_df["Type"].replace(category_mapping)

<h3>4.5 Filtering and Saving Data for Santo André</h3>

**Note:**  
The following code filters the places to include only those in the city of "Santo André".  
This code has not been executed yet, but it is available if needed in the future.

In [ ]:
# Filter the DataFrame to include only places in the city of "Santo André"  
places_df = places_df[places_df["City"] == "Santo André"]  

In [ ]:
# Save the data to a CSV file
places_df.to_csv(os.path.join(os.path.abspath(".."), "data/intermediate/places_filtered.csv"), sep=";", index=False, encoding="utf-8")

<h3>4.6 Filtering Non-Pet Shop Locations</h3>

**Note:**  
Manually review the excluded_places.csv file and keep only the places to be excluded.

In [ ]:
# List of keywords that typically appear in pet shop names
petshop_keywords = ["pet", "shop", "store", "veterinária", "vet", "animal", "animais", "dog", "cachorro", "cão", "cães", "cat", "gato", 
                    "grooming", "groomer", "banho", "tosa", "agropecuária", "avicultura", "pássaros", "aquário", "aquarismo", "aquarium",
                    "peixes", "ornamental", "reef", "casa de racao", "ração", "rações", "bicho", "bichos", "bichinho", "pata", "patinhas", 
                    "puppy", "filhote", "criadouro", "focinho", "late", "mia", "miau", "zoo", "amigo", "toca"]

In [ ]:
# Filter locations that DO NOT contain pet shop-related keywords
df_excluded = places_df[~places_df["Name"].astype(str).apply(lambda x: any(p in x.lower() for p in petshop_keywords))]

In [ ]:
# Save the excluded locations to a CSV file
df_excluded.to_csv(os.path.join(os.path.abspath(".."), "data/intermediate/excluded_places.csv"), sep=";", index=False, encoding="utf-8")

<h3>4.7 Excluding Incorrect Places from DataFrame</h3>

In [ ]:
# Load the list of incorrect places from a CSV file  
EXCLUDED_PLACES_PATH = os.path.abspath(os.path.join("..", "data", "intermediate", "excluded_places.csv"))  
excluded_places = pd.read_csv(EXCLUDED_PLACES_PATH, sep=";", header=0, encoding="utf-8")["Place ID"].tolist()

In [ ]:
# Remove these places from the main DataFrame  
places_df = places_df[~places_df["Place ID"].isin(excluded_places)]  

<h2>5. Export Processed Data</h2>

In [ ]:
# Save the data to a CSV file
places_df.to_csv(os.path.join(os.path.abspath(".."), "data/processed/places_processed.csv"), sep=";", index=False, encoding="utf-8")